# Goal 
Lets look at an example of how to train a model using `torch.compile` and  measure the performance improvements we observe

## Model & DataSet
We will train a ResNet model to classify cats and dogs

We use `datatsets` from [HuggingFace](https://huggingface.co/docs/datasets/index) to load the cats and dogs dataset
```
pip install -r requirements.txt
```

# Hardware

The current notebook has been run on Nvidia A10 GPU. 
Please adjust the batch_size based on the GPU being used

# Measuring Performance

To measure performance gains with `torch.compile`, we write a helper function

```python
def timed(fn):
    start = torch.cuda.Event(enable_timing=True)
    end = torch.cuda.Event(enable_timing=True)
    start.record()
    result = fn()
    torch.cuda.synchronize()
    end.record()
    return result, start.elapsed_time(end) / 1000
```

We call the `train` and `test` functions with the `timed` function

As a first step, we train the model without `torch.compile` and measure performance

# Train the model without torch.compile

In [3]:
!python main.py

Found cached dataset cats_vs_dogs (/home/ubuntu/.cache/huggingface/datasets/cats_vs_dogs/default/1.0.0/d4fe9cf31b294ed8639aa58f7d8ee13fe189011837038ed9a774fde19a911fcb)
Using ResNet18
torch.compile() disabled
Train Epoch: 1 [0/18728 (0%)]	Loss: 0.772188
Train Epoch: 1 [12800/18728 (68%)]	Loss: 0.112535
Training Time: 45.50344921875

Test set: Average loss: 0.0000, Accuracy: 4622/4682 (99%)

Evaluation Time: 17.61987109375
#########################################################
Train Epoch: 2 [0/18728 (0%)]	Loss: 0.139746
Train Epoch: 2 [12800/18728 (68%)]	Loss: 0.079648
Training Time: 41.850609375

Test set: Average loss: 0.0000, Accuracy: 4631/4682 (99%)

Evaluation Time: 14.20183203125
#########################################################
Train Epoch: 3 [0/18728 (0%)]	Loss: 0.112746
Train Epoch: 3 [12800/18728 (68%)]	Loss: 0.047305
Training Time: 41.91703515625

Test set: Average loss: 0.0000, Accuracy: 4628/4682 (99%)

Evaluation Time: 14.158279296875
#########################

We observe that the training time is around 41.9 seconds

# Train the model with torch.compile

In [10]:
!python main.py --torch-compile

Found cached dataset cats_vs_dogs (/home/ubuntu/.cache/huggingface/datasets/cats_vs_dogs/default/1.0.0/d4fe9cf31b294ed8639aa58f7d8ee13fe189011837038ed9a774fde19a911fcb)
Using ResNet18
torch.compile() enabled
Train Epoch: 1 [0/18728 (0%)]	Loss: 0.757062
Train Epoch: 1 [12800/18728 (68%)]	Loss: 0.156891
Training Time: 63.32085546875

Test set: Average loss: 0.0000, Accuracy: 4610/4682 (98%)

Evaluation Time: 22.271330078125
#########################################################
Train Epoch: 2 [0/18728 (0%)]	Loss: 0.102807
Train Epoch: 2 [12800/18728 (68%)]	Loss: 0.098924
Training Time: 42.25797265625

Test set: Average loss: 0.0000, Accuracy: 4619/4682 (99%)

Evaluation Time: 13.9923232421875
#########################################################
Train Epoch: 3 [0/18728 (0%)]	Loss: 0.059663
Train Epoch: 3 [12800/18728 (68%)]	Loss: 0.075105
Training Time: 42.20574609375

Test set: Average loss: 0.0000, Accuracy: 4624/4682 (99%)

Evaluation Time: 14.1473486328125
####################

### We observe that the training time is around 42 seconds whether we use `torch.compile` or not

We also observe that the first epoch is slower. That is expected and can be ignored for comparison purposes

## What's happening? Why is torch.compile not speeding up the model training?

Lets profile the forward pass, backward pass and data loading times to see what's happening

In [11]:
!python main.py --torch-compile --profile

Found cached dataset cats_vs_dogs (/home/ubuntu/.cache/huggingface/datasets/cats_vs_dogs/default/1.0.0/d4fe9cf31b294ed8639aa58f7d8ee13fe189011837038ed9a774fde19a911fcb)
Using ResNet18
torch.compile() enabled
Train Epoch: 1 [0/18728 (0%)]	Loss: 0.860411
Median forward time (ms) 8591.79 | backward time (ms) 3444.59 | dataloader time (ms) 291.37
Train Epoch: 1 [12800/18728 (68%)]	Loss: 0.132526
Median forward time (ms) 40.39 | backward time (ms) 71.37 | dataloader time (ms) 275.78
Total forward time (s) 20.09 | backward time (s) 16.65 | dataloader time (s) 40.32
Training Time: 77.14421875

Test set: Average loss: 0.0000, Accuracy: 4610/4682 (98%)

Evaluation Time: 22.210673828125
#########################################################
Train Epoch: 2 [0/18728 (0%)]	Loss: 0.142593
Median forward time (ms) 41.04 | backward time (ms) 71.45 | dataloader time (ms) 266.17
Train Epoch: 2 [12800/18728 (68%)]	Loss: 0.056003
Median forward time (ms) 40.44 | backward time (ms) 71.41 | dataloader ti

We observe that dataloading is the bottleneck. Dataloading has a median time of 270 ms vs Forward pass of 40ms

Current Dataloader config

```python
train_kwargs = {'batch_size': args.batch_size, 'shuffle': True}
train_loader = torch.utils.data.DataLoader(train_ds,**train_kwargs)
```

Lets add the following args to the DataLoader
```python
opt_kwargs = {'num_workers': 4,
              'pin_memory': True}
        train_kwargs.update(opt_kwargs)
```

You can read more about memory pinning [here](https://pytorch.org/docs/stable/data.html#memory-pinning)

Now we train the model with the dataloader optimizations

# Train the model without torch.compile and DataLoader Optimizations

In [3]:
!python main.py --dl-opt

Found cached dataset cats_vs_dogs (/home/ubuntu/.cache/huggingface/datasets/cats_vs_dogs/default/1.0.0/d4fe9cf31b294ed8639aa58f7d8ee13fe189011837038ed9a774fde19a911fcb)
Using ResNet18
torch.compile() disabled
Train Epoch: 1 [0/18728 (0%)]	Loss: 0.837809
Train Epoch: 1 [12800/18728 (68%)]	Loss: 0.206172
Training Time: 21.717208984375

Test set: Average loss: 0.0000, Accuracy: 4622/4682 (99%)

Evaluation Time: 10.4892626953125
#########################################################
Train Epoch: 2 [0/18728 (0%)]	Loss: 0.076245
Train Epoch: 2 [12800/18728 (68%)]	Loss: 0.099027
Training Time: 18.212275390625

Test set: Average loss: 0.0000, Accuracy: 4625/4682 (99%)

Evaluation Time: 6.2697470703125
#########################################################
Train Epoch: 3 [0/18728 (0%)]	Loss: 0.078262
Train Epoch: 3 [12800/18728 (68%)]	Loss: 0.042700
Training Time: 18.23256640625

Test set: Average loss: 0.0000, Accuracy: 4628/4682 (99%)

Evaluation Time: 6.308853515625
###################

# Train the model with torch.compile and DataLoader Optimizations

In [4]:
!python main.py --torch-compile --dl-opt

Found cached dataset cats_vs_dogs (/home/ubuntu/.cache/huggingface/datasets/cats_vs_dogs/default/1.0.0/d4fe9cf31b294ed8639aa58f7d8ee13fe189011837038ed9a774fde19a911fcb)
Using ResNet18
torch.compile() enabled
Train Epoch: 1 [0/18728 (0%)]	Loss: 0.736574
Train Epoch: 1 [12800/18728 (68%)]	Loss: 0.090557
Training Time: 86.2654453125

Test set: Average loss: 0.0000, Accuracy: 4614/4682 (99%)

Evaluation Time: 16.06041015625
#########################################################
Train Epoch: 2 [0/18728 (0%)]	Loss: 0.164315
Train Epoch: 2 [12800/18728 (68%)]	Loss: 0.064604
Training Time: 16.964841796875

Test set: Average loss: 0.0000, Accuracy: 4625/4682 (99%)

Evaluation Time: 6.28181396484375
#########################################################
Train Epoch: 3 [0/18728 (0%)]	Loss: 0.106078
Train Epoch: 3 [12800/18728 (68%)]	Loss: 0.113204
Training Time: 16.94748046875

Test set: Average loss: 0.0000, Accuracy: 4622/4682 (99%)

Evaluation Time: 6.27323193359375
#####################

### Now we see that the training time per epoch has reduced from 18.2 seconds to 16.9 seconds (7% speedup)

# torch.compile(mode="reduce-overhead")

We use `reduce-overhead` when we have smaller batch size or small sequence lengths in case of language models. Basically, if operations are more CPU bound, using this mode will speed up training

# Train the model without torch.compile and batch-size 8

In [19]:
!python main.py --dl-opt --batch-size 8

Found cached dataset cats_vs_dogs (/home/ubuntu/.cache/huggingface/datasets/cats_vs_dogs/default/1.0.0/d4fe9cf31b294ed8639aa58f7d8ee13fe189011837038ed9a774fde19a911fcb)
Using ResNet18
torch.compile() disabled
Train Epoch: 1 [0/18728 (0%)]	Loss: 0.698328
Train Epoch: 1 [800/18728 (4%)]	Loss: 0.191970
Train Epoch: 1 [1600/18728 (9%)]	Loss: 0.369378
Train Epoch: 1 [2400/18728 (13%)]	Loss: 0.098166
Train Epoch: 1 [3200/18728 (17%)]	Loss: 0.046861
Train Epoch: 1 [4000/18728 (21%)]	Loss: 0.043571
Train Epoch: 1 [4800/18728 (26%)]	Loss: 0.679784
Train Epoch: 1 [5600/18728 (30%)]	Loss: 0.877805
Train Epoch: 1 [6400/18728 (34%)]	Loss: 0.137903
Train Epoch: 1 [7200/18728 (38%)]	Loss: 0.062516
Train Epoch: 1 [8000/18728 (43%)]	Loss: 0.048021
Train Epoch: 1 [8800/18728 (47%)]	Loss: 0.603960
Train Epoch: 1 [9600/18728 (51%)]	Loss: 0.071718
Train Epoch: 1 [10400/18728 (56%)]	Loss: 0.049384
Train Epoch: 1 [11200/18728 (60%)]	Loss: 0.164265
Train Epoch: 1 [12000/18728 (64%)]	Loss: 0.156786
Train Epoch

# Train the model with torch.compile and batch-size 8

In [17]:
!python main.py --torch-compile --dl-opt --batch-size 8

Found cached dataset cats_vs_dogs (/home/ubuntu/.cache/huggingface/datasets/cats_vs_dogs/default/1.0.0/d4fe9cf31b294ed8639aa58f7d8ee13fe189011837038ed9a774fde19a911fcb)
Using ResNet18
torch.compile() enabled
Train Epoch: 1 [0/18728 (0%)]	Loss: 0.738092
Train Epoch: 1 [800/18728 (4%)]	Loss: 0.004603
Train Epoch: 1 [1600/18728 (9%)]	Loss: 0.062418
Train Epoch: 1 [2400/18728 (13%)]	Loss: 0.410624
Train Epoch: 1 [3200/18728 (17%)]	Loss: 0.190926
Train Epoch: 1 [4000/18728 (21%)]	Loss: 0.125716
Train Epoch: 1 [4800/18728 (26%)]	Loss: 0.238159
Train Epoch: 1 [5600/18728 (30%)]	Loss: 0.149272
Train Epoch: 1 [6400/18728 (34%)]	Loss: 0.524148
Train Epoch: 1 [7200/18728 (38%)]	Loss: 0.246480
Train Epoch: 1 [8000/18728 (43%)]	Loss: 0.377747
Train Epoch: 1 [8800/18728 (47%)]	Loss: 0.280981
Train Epoch: 1 [9600/18728 (51%)]	Loss: 0.137405
Train Epoch: 1 [10400/18728 (56%)]	Loss: 0.187106
Train Epoch: 1 [11200/18728 (60%)]	Loss: 0.042297
Train Epoch: 1 [12000/18728 (64%)]	Loss: 0.546959
Train Epoch:

We notice that training time per epoch has increased from 28 seconds to 30 seconds in case of small batch sizes

Lets train with `reduce-overhead` mode

In [18]:
!python main.py --torch-compile --reduce-overhead --dl-opt --batch-size 8

Found cached dataset cats_vs_dogs (/home/ubuntu/.cache/huggingface/datasets/cats_vs_dogs/default/1.0.0/d4fe9cf31b294ed8639aa58f7d8ee13fe189011837038ed9a774fde19a911fcb)
Using ResNet18
torch.compile() enabled
Mode 'reduce-overhead' enabled
Train Epoch: 1 [0/18728 (0%)]	Loss: 1.028874
Train Epoch: 1 [800/18728 (4%)]	Loss: 0.652823
Train Epoch: 1 [1600/18728 (9%)]	Loss: 0.753654
Train Epoch: 1 [2400/18728 (13%)]	Loss: 0.070327
Train Epoch: 1 [3200/18728 (17%)]	Loss: 0.018217
Train Epoch: 1 [4000/18728 (21%)]	Loss: 1.530392
Train Epoch: 1 [4800/18728 (26%)]	Loss: 0.056403
Train Epoch: 1 [5600/18728 (30%)]	Loss: 0.237657
Train Epoch: 1 [6400/18728 (34%)]	Loss: 0.267497
Train Epoch: 1 [7200/18728 (38%)]	Loss: 0.174963
Train Epoch: 1 [8000/18728 (43%)]	Loss: 0.057229
Train Epoch: 1 [8800/18728 (47%)]	Loss: 0.006394
Train Epoch: 1 [9600/18728 (51%)]	Loss: 2.001338
Train Epoch: 1 [10400/18728 (56%)]	Loss: 0.376467
Train Epoch: 1 [11200/18728 (60%)]	Loss: 0.044305
Train Epoch: 1 [12000/18728 (64

We notice that training time per epoch reduces from 28 seconds to 25 seconds

# Train ResNet152 model

Lets look at how much performance improvement we get when we train a bigger model

# Train the model without torch.compile and batch-size 64

In [2]:
!python main.py  --dl-opt --resnet152 --batch-size 64

Found cached dataset cats_vs_dogs (/home/ubuntu/.cache/huggingface/datasets/cats_vs_dogs/default/1.0.0/d4fe9cf31b294ed8639aa58f7d8ee13fe189011837038ed9a774fde19a911fcb)
Using ResNet152
torch.compile() disabled
Train Epoch: 1 [0/18728 (0%)]	Loss: 0.674462
Train Epoch: 1 [6400/18728 (34%)]	Loss: 0.214665
Train Epoch: 1 [12800/18728 (68%)]	Loss: 0.094949
Training Time: 134.3393125

Test set: Average loss: 0.0000, Accuracy: 4636/4682 (99%)

Evaluation Time: 24.82583984375
#########################################################
Train Epoch: 2 [0/18728 (0%)]	Loss: 0.072032
Train Epoch: 2 [6400/18728 (34%)]	Loss: 0.056706
Train Epoch: 2 [12800/18728 (68%)]	Loss: 0.076511
Training Time: 129.023953125

Test set: Average loss: 0.0000, Accuracy: 4642/4682 (99%)

Evaluation Time: 14.32539453125
#########################################################
Train Epoch: 3 [0/18728 (0%)]	Loss: 0.069018
Train Epoch: 3 [6400/18728 (34%)]	Loss: 0.055646
Train Epoch: 3 [12800/18728 (68%)]	Loss: 0.073438
Tr

# Train the model with torch.compile and batch-size 64

In [1]:
!python main.py --torch-compile  --dl-opt --resnet152 --batch-size 64

Found cached dataset cats_vs_dogs (/home/ubuntu/.cache/huggingface/datasets/cats_vs_dogs/default/1.0.0/d4fe9cf31b294ed8639aa58f7d8ee13fe189011837038ed9a774fde19a911fcb)
Using ResNet152
torch.compile() enabled
Train Epoch: 1 [0/18728 (0%)]	Loss: 0.678646
Train Epoch: 1 [6400/18728 (34%)]	Loss: 0.169159
Train Epoch: 1 [12800/18728 (68%)]	Loss: 0.089722
Training Time: 362.74515625

Test set: Average loss: 0.0000, Accuracy: 4648/4682 (99%)

Evaluation Time: 57.85243359375
#########################################################
Train Epoch: 2 [0/18728 (0%)]	Loss: 0.090307
Train Epoch: 2 [6400/18728 (34%)]	Loss: 0.091355
Train Epoch: 2 [12800/18728 (68%)]	Loss: 0.210980
Training Time: 116.170125

Test set: Average loss: 0.0000, Accuracy: 4658/4682 (99%)

Evaluation Time: 11.5616328125
#########################################################
Train Epoch: 3 [0/18728 (0%)]	Loss: 0.097246
Train Epoch: 3 [6400/18728 (34%)]	Loss: 0.037607
Train Epoch: 3 [12800/18728 (68%)]	Loss: 0.083593
Traini

### Now we see that the training time per epoch has reduced from 129 seconds to 116 seconds (11% speedup)

### We see  more speedup when the model is bigger